In [1]:
# !pip install se3-transformer-pytorch
# !pip install h5py

In [2]:
import os
import numpy as np
import h5py
import torch
from se3_transformer_pytorch import SE3Transformer
from tqdm import tqdm

In [3]:
def load_hdf5_data(file_path):
    with h5py.File(file_path, 'r') as h5f:
        coords = h5f['coords'][:]
        features = h5f['features'][:]
        labels = h5f['labels'][:]
    return coords, features, labels

# Load training and test datasets
train_file_path = 'prepared_dataset_1024.h5'  # Update with your training dataset path
test_file_path = 'datatest_1024.h5' # Update with your test dataset path

train_coords, train_features, train_labels = load_hdf5_data(train_file_path)
test_coords, test_features, test_labels = load_hdf5_data(test_file_path)

In [ ]:
# Define the SE3 Transformer model

feature_dim = train_features.shape[-1]  # Assuming train_features is a numpy array

model = SE3Transformer(
    dim=feature_dim,
    heads=8,
    depth=6,
    dim_head=64,
    num_degrees=4,
)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

TypeError: SE3Transformer.__init__() got an unexpected keyword argument 'neighbors'

In [5]:
# Define training parameters
num_epochs = 100
learning_rate = 1e-4
batch_size = 1  # Adjust based on your GPU memory

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for i in tqdm(range(len(train_features))):
        # Prepare input data
        feats = torch.tensor(train_features[i]).unsqueeze(0).to(device)  # Shape: (1, num_points, feature_dim)
        coors = torch.tensor(train_coords[i], dtype=torch.float32).unsqueeze(0).to(device)  # Shape: (1, num_points, 3)
        mask = torch.ones(feats.shape[0], feats.shape[1]).bool().to(device)  # Shape: (1, num_points)

        # Ensure the coordinates tensor has the correct shape
        if coors.dim() == 2:
            coors = coors.unsqueeze(0)  # Shape: (1, num_points, 3)

        # Forward pass
        outputs = model(feats, coors, mask)  # Shape: (1, num_points, feature_dim)

        # Calculate loss
        label = torch.tensor(train_labels[i]).unsqueeze(0).to(device)  # Shape: (1,)
        loss = criterion(outputs.view(-1, outputs.size(-1)), label.view(-1))
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_features):.4f}')


  0%|          | 0/153600 [00:00<?, ?it/s]


AssertionError: you must be fetching at least 1 neighbor